In [0]:
%run "/Workspace/Users/anirudhp@megnity.com/healthcare_project/src/Explore and Clean"

In [0]:
first_camp_cleaned.printSchema()
second_camp_cleaned.printSchema()
third_camp_cleaned.printSchema()
patient_profiles_cleaned.printSchema()
health_camp_details_cleaned.printSchema()

### Custom framework to plan the transformations to be done 

1. There are 5 tables. See if there is any relationship between them. 
2. If there is a relationship, check if it makes sense to put the related tables together in whichever way it can be required for end user usecase
3. If needed, create new calculated or derived field if needed
4. The end outcome could be unified table or tables

### The transformations that'll be done are the following:

1. Combine the patient_id and health_camp_id from the camp tables into one dataframe. Then create a new column called camp_format and identify which format the unique combination of patient_id and health_camp_id falls under.
2. Add the health scores for each unique combination of patient_id and health_camp_id. Note: the health score is only provided to people who attented either or both of the first two formats. 
3. Add other information such as donation for the people who attended the first camp format, stalls visited from the third camp format and other patient details from the table patient_profiles.
4. Create a derived column of all the social media followers a patient has by totalling the followers they have across all the platforms they are in. Then remove the individual platform followers count columns. 



In [0]:
# Union attendance DataFrames with Patient_ID, Health_Camp_ID, and Camp_Format
try:
    df_first_selected = first_camp_cleaned.select(
        col("patient_id"),
        col("health_camp_id"),
        lit("First").alias("Camp_Format")
    )
    df_second_selected = second_camp_cleaned.select(
        col("patient_id"),
        col("health_camp_id"),
        lit("Second").alias("Camp_Format")
    )
    df_third_selected = third_camp_cleaned.select(
        col("patient_id"),
        col("health_camp_id"),
        lit("Third").alias("Camp_Format")
    )

    # Union the DataFrames
    all_patients = df_first_selected.union(df_second_selected).union(df_third_selected)
    print("Attendance DataFrames unified successfully.")
except Exception as e:
    print(f"Error during union: {e}")
    raise

display(all_patients)

In [0]:
# Combine the health scores from the first_camp_cleaned and second_camp_cleaned

combined_health_score = first_camp_cleaned.select(
    first_camp_cleaned.columns[0],
    first_camp_cleaned.columns[1],
    first_camp_cleaned.columns[3]
).union(
    second_camp_cleaned.select(second_camp_cleaned.columns[:3])
)

# add all the health scores available to our all_patients table

all_patients = all_patients.join(
    combined_health_score.select("Patient_ID", "Health_Camp_ID", "health_score"),
    ["Patient_ID", "Health_Camp_ID"],
    "left"
)

display(all_patients)

In [0]:
# add the donation information from first_camp_cleaned into all_patients

all_patients = all_patients.join(
    first_camp_cleaned.select(
        col("Patient_ID"),
        col("Health_Camp_ID"),
        col("donation")
    ),
    ["Patient_ID", "Health_Camp_ID"],
    "left"
    )
    
display(all_patients)

In [0]:
# add the number of stall visited and last stall visited number information from the third_camp_cleaned

all_patients = all_patients.join(
    third_camp_cleaned.select(
        col("Patient_ID"),
        col("Health_Camp_ID"),
        col("number_of_stall_visited"),
        col("last_stall_visited_number")
    ),
    ["Patient_ID", "Health_Camp_ID"],
    "left"
)

display(all_patients.filter(col("Camp_Format") == "Third"))

In [0]:
# Join with patient_profile_clean for the rest of the patients' details

all_patient_details = all_patients.join(
    patient_profiles_cleaned,
    "Patient_ID",
    "left"
)

display(all_patient_details.limit(25))

In [0]:
# combine the columns online_follower, linkedin_shared, twitter_shared, facebook_shared into one column called 'total_social_media_followers' by adding the values horizontally. Then drop the columns online_follower, linkedin_shared, twitter_shared, facebook_shared

all_patient_details = all_patient_details.withColumn(
    "total_social_media_followers",
    col("online_follower") + col("linkedin_shared") + col("twitter_shared") + col("facebook_shared")
).drop(
    "online_follower",
    "linkedin_shared",
    "twitter_shared",
    "facebook_shared"
)

display(all_patient_details.limit(25))

In [0]:
from pyspark.sql.functions import col, count, when, round

cols_to_check = ["income", "education_score", "age", "first_interaction", "city_type", "employer_category"]
total_rows = all_patient_details.count()

null_counts = all_patient_details.select([
    count(when(col(c).isNull(), c)).alias(f"{c}_null_count") for c in cols_to_check
])

null_percentages = null_counts.select([
    (col(f"{c}_null_count") / total_rows * 100).alias(f"{c}_null_pct") for c in cols_to_check
])


display(null_percentages)

In [0]:
# How do I deal with the null values in the columns income, education_score, age, city_type, employer_category?

# We could consider dropping columns with a high null value percentage but because we have a large enough dataset, even 20% of non-null data can be useful for all the above columns. So dropping columns here may not be the best approach. 

# Similarly if we need to consider dropping rows, the only columns that are critical would be the patient_id and health_camp_id columns which we have 0 null values in. So dropping rows here may not be the best approach either.

# If we need to consider imputing values, ideally the null % should be quite less like 15% or less. So imputing values here may not be the best approach either.

# Because it doesn't make clear sense to either drop rows/columns or impute values in our all_patient_details dataframe, we could keep the columns as it is and during analysis consider the null percentages and work with the dataframe as needed. 


# However, for the purposes of learning imputing, we will perform it here:
'''
- impute the null values in income with its median value. Income data is usually skewed, therefore using median would be better. Median is not affected by outliers.

- impute the null values in education_score with its mean value. Education_score calculation hasn't been defined in the data description. Hence, we will assume that the data isn't skewed and will use mean. 

- impute the null values in age with its mean value. Generally, in some places you will see data being skewed in one direction for age. For example, Japan has a population with a lot of people in the older age group. Another example, would be India, where the population is skewed towards the younger age group. So if you had to randomly pick an individual from Japan, an educated guess would be that the person would be older. However, if you had to randomly pick an individual from India, an educated guess would be that the person would be younger. Hence, we will use mean for imputing age.

- impute the null values in city_type with its mode value. The values in the city_type is categorical and we don't have much information to work with in the data description. So we will use mode for imputing city_type which is usually what is used for categorical data.

- impute the null values in employer_category with the value 'Others'. Because we have very few values to work with and there is no one category where there is way too many people, we will fill it 'Others' for the purpose of this project

- impute the null values in health_score with its mean value. Ideally the better thing to do for health_score would be to create a regression model and make it impute the values. But for the purposes of building an end-to-end data pipeline in practice, we will skip it for now. 

- impute the null values in donation with its median value. Mean would skewed similar to income. 

- impute the null values with 0 for number_of_stall_visited and last_stall_visited_number. This would be true because according to the data description, only the people who attended the third health camp format, visited stalls. It would make sense that others who never attended the third health camp format, would have visited no stalls.
'''

In [0]:
# impute the null values in income with its median value

income_median = all_patient_details.approxQuantile("income", [0.5], 0.01)[0]
all_patient_details = all_patient_details.fillna(income_median, subset=["income"])


In [0]:
from pyspark.sql.functions import avg

# impute the null values in education_score with its mean value

education_score_mean = all_patient_details.agg(avg("education_score")).collect()[0][0]
all_patient_details = all_patient_details.fillna(education_score_mean, subset=["education_score"])

In [0]:
# impute the null values in age with its mean value.

age_mean = all_patient_details.agg(avg("age")).collect()[0][0]
all_patient_details = all_patient_details.fillna(age_mean, subset=["age"])

In [0]:
# impute the null values in city_type with its mode value

city_type_mode = all_patient_details.agg(mode("city_type")).collect()[0][0]
all_patient_details = all_patient_details.fillna(city_type_mode, subset=["city_type"])

In [0]:
# impute the null values in employer_category with the value 'Others'

all_patient_details = all_patient_details.fillna("Others", subset=["employer_category"])

In [0]:
# impute the null values in health_score with its mean value

health_score_mean = all_patient_details.agg(avg("health_score")).collect()[0][0]
all_patient_details = all_patient_details.fillna(health_score_mean, subset=["health_score"])

In [0]:
# impute the null values in donation with its median value

donation_median = all_patient_details.approxQuantile("donation", [0.5], 0.01)[0]
all_patient_details = all_patient_details.fillna(donation_median, subset=["donation"])

In [0]:
# impute the null values with 0 for number_of_stall_visited and last_stall_visited_number

all_patient_details = all_patient_details.fillna(0, subset=["number_of_stall_visited", "last_stall_visited_number"])